# Part II: MOFA

## 1. Load the required packages

## 2. Load the data of the previous step

## 3. Run (and save) MOFA

### 3.1 Show diagnostic plots

## 4. Covariate analysis

## 5. Explore results

In [ ]:
MOFAobject <- create_mofa(mae.final)

plot_data_overview(MOFAobject)

data_opts <- get_default_data_options(MOFAobject)
data_opts

model_opts <- get_default_model_options(MOFAobject)
model_opts$num_factors <- 15

train_opts <- get_default_training_options(MOFAobject)
train_opts$convergence_mode <- "fast"

MOFAobject <- prepare_mofa(MOFAobject,
  data_options = data_opts,
  model_options = model_opts,
  training_options = train_opts
)

# Create model file for loading the experiment in previous steps
outfile = file.path(getwd(), "../MOFAobject.LUAD.hdf5")
MOFAobject.trained <- run_mofa(MOFAobject, outfile, use_basilisk = FALSE)
views_names(MOFAobject.trained) <- c("CNV","Protein","RNA")

In [ ]:
serialized_mae_path = file.path(getwd(), "../mae.rds")
mae.final <- readRDS(serialized_mae_path)

In [ ]:
# Plot the individual top weights on selected Factors regarding specific layers
plot_top_weights(MOFAobject.trained,
	view = "RNA",
	factor = 1, 
	nfeatures = 10,    
	scale = T)

# Plot all samples across factors
plot_factor(MOFAobject.trained, 
	factor = 1:15,
	color_by = "gender",
	shape_by = "vital_status")

# Example violin plot (factor 3 & gender)
violin_plot <- plot_factor(MOFAobject.trained,factor=3, color_by="gender", dodge = T, add_violin=T, add_boxplot=F)
violin_plot <- violin_plot + ggpubr::stat_compare_means()
plot(violin_plot)

# Example scatter plots (top10 expressed genes & pathologic stage)
plot_data_scatter(MOFAobject.trained, 
  view = "RNA",
  factor = 1,  
  features = 10,
  sign = "positive",
  color_by = "pathologic_stage"
) + labs(y="RNA expression")

# Visualization of combinations of factors
plot_factors(MOFAobject.trained,
	factors = 1:5,
	color_by = "pathology_T_stage"
)

# Visualization of feature weights
plot_weights(MOFAobject.trained,
view = "RNA",
factor = 1,
# Number of features to highlight
nfeatures = 10, 
# Scale weights from -1 to 1
scale = TRUE, 
# Take the absolute value?
abs = FALSE 
)

# Heatmaps
plot_data_heatmap(MOFAobject.trained,
view = "RNA", # view of interest
factor = 1, 
features = 20, 
cluster_rows = TRUE, cluster_cols = FALSE,
show_rownames = TRUE, show_colnames = FALSE,
color = brewer.pal(n = 7, name = "Blues")
)

In [ ]:
library(MOFA2)
library(reticulate)
library(data.table)
library(GGally)
library(RColorBrewer)

# R needs to interact with Python. We prepared everything - no need to worry about this command.
reticulate::use_python("/opt/conda/bin/python", required=TRUE)

In [ ]:
plot_factor_cor(MOFAobject.trained)
plot_variance_explained(MOFAobject.trained, max_r2=10)
plot_variance_explained(MOFAobject.trained, plot_total = T)[[2]]

In [ ]:
covariates <- colnames(samples_metadata(MOFAobject.trained))

# Remove "group","patientID", and "sample" from covariates list
correlate_factors_with_covariates(MOFAobject.trained, covariates = covariates[-c(3,10,12)], plot="log_pval")